In [3]:
!pip3 install influxdb-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 706 kB 4.9 MB/s 
     |████████████████████████████████| 140 kB 48.5 MB/s 
     |████████████████████████████████| 217 kB 53.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.12 which is incompatible.


In [1]:
## importando as dependências
import tensorflow as tf 
import numpy as np
import pandas as pd

## Loading the data as a pandas dataframe 
data = pd.read_csv("https://raw.githubusercontent.com/gouravsinghbais/Time-Series-Forecasting-with-Tensorflow-and-InfluxDB/master/sunspots-dataset/Sunspots.csv", index_col=0) 
## show first dew rows of the dataset
data.head()

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


,Date,Monthly Mean Total Sunspot Number
0,1749-01-31,96.7
1,1749-02-28,104.3
2,1749-03-31,116.7
3,1749-04-30,92.8
4,1749-05-31,141.7


In [2]:
## importando as dependências
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS


token = ""
org = ""
bucket = ""
url=""
## conectando ao InfluxDB
client = InfluxDBClient(url=url , token=token, org = org)

In [3]:
# convertendo coluna Date para datetime
data['Date'] = pd.to_datetime(data['Date'])

## cria data como índice
data.set_index(data['Date'], drop = True, inplace = True)
data.drop('Date', axis = 1, inplace = True)
data.head()

,Monthly Mean Total Sunspot Number
Date,
1749-01-31,96.7
1749-02-28,104.3
1749-03-31,116.7
1749-04-30,92.8
1749-05-31,141.7


In [4]:
## cria objeto de escrita da API
## SYNCHRONOUS: armazena múltiplos registros de data por vez
write_api = client.write_api(write_options=SYNCHRONOUS)

In [5]:
## armazena dados no influxdb
## write data to influxdb
response = write_api.write(bucket, org=org, record = data, data_frame_measurement_name='sunspot',
                    	data_frame_tag_columns=['sunspot'])

In [ ]:
## busca os dados (utiliza linguagem Flux)
## Recupera dos dados dos últimos 275 anos
query_api = client.query_api()
tables = query_api.query('from(bucket:"teste") |> range(start: -275y)')